In [1]:
# here I prepare the data for Machine Learning

In [2]:
# this is a class that can be used to easuly import datasets from yahoo finance using panda datareaser.
# 
import numpy as np

# Create the class
class ImportData():

    # Initialize the class
    def __init__(self, symbol, start_date, end_date):
        self.df = self._extract_data(symbol, start_date, end_date)
        self.sharpe = 0

    # Extract data
    def _extract_data(self, symbol, start_date, end_date):
        from pandas_datareader.data import DataReader
        data = DataReader(name=symbol, data_source='yahoo', start=start_date, end=end_date)
        data = data[["Open", "High", "Low", "Close", "Volume"]]
        data = self._structure_df(data)
        return data

    # Calculates general period returns and volatility
    def _structure_df(self, df):
        df["Returns"] = df["Close"].pct_change()
        df["Range"] = df["High"] / df["Low"] - 1
        df["Bench_C_Rets"], sharpe = self._calculate_returns(df, True)
        self.sharpe = sharpe
        df.dropna(inplace=True)
        return df

    # Adjusts the signal to represent our strategy
    def _set_multiplier(self, direction):
        if direction == "long":
            pos_multiplier = 1
            neg_multiplier = 0
        elif direction == "long_short":
            pos_multiplier = 1
            neg_multiplier = -1
        else:
            pos_multiplier = 0
            neg_multiplier = -1
        return pos_multiplier, neg_multiplier

    # Calculates returns for equity curve
    def _calculate_returns(self, df, is_benchmark):
        
        # Calculate multiplier
        if not is_benchmark:
            multiplier_1 = df["Signal"]
            multiplier_2 = 1 if "PSignal" not in df.columns else df["PSignal"]
            log_rets = np.log(df["Close"] / df["Close"].shift(1)) * multiplier_1 * multiplier_2
        else:
            multiplier_1 = 1
            multiplier_2 = 1
            
            # Assume open price on following day to avoid lookahead bias for close calculation
            log_rets = np.log(df["Open"].shift(-1) / df["Close"].shift(1)) * multiplier_1 * multiplier_2
        
        # Calculate Sharpe Ratio
        sharpe_ratio = self.sharpe_ratio(log_rets)
        
        # Calculate Cumulative Returns
        c_log_rets = log_rets.cumsum()
        c_log_rets_exp = np.exp(c_log_rets) - 1
        
        # Return result and Sharpe ratio
        return c_log_rets_exp, sharpe_ratio
    
    def sharpe_ratio(self, return_series):
        N = 255 # Trading days in the year (change to 365 for crypto)
        rf = 0.005 # Half a percent risk free rare
        mean = return_series.mean() * N -rf
        sigma = return_series.std() * np.sqrt(N)
        sharpe = round(mean / sigma, 3)
        return sharpe

    # Replace Dataframe
    def change_df(self, new_df, drop_cols=[]):
        new_df = new_df.drop(columns=drop_cols)
        self.df = new_df

    # Moving average crossover strategy
    def backtest_ma_crossover(self, period_1, period_2, direction, drop_cols=[]):
        
        # Set df
        df = self.df
        
        # Get multipliers
        pos_multiplier, neg_multiplier = self._set_multiplier(direction)
            
        # Calculate Moving Averages
        if f"MA_{period_1}" or f"MA_{period_2}" not in df.columns:
            df[f"MA_{period_1}"] = df["Close"].rolling(window=period_1).mean()
            df[f"MA_{period_2}"] = df["Close"].rolling(window=period_2).mean()
            df.dropna(inplace=True)
        
        # Calculate Benchmark Returns
        df["Bench_C_Rets"], sharpe_ratio_bench = self._calculate_returns(df, True)
        
        # Calculate Signal
        df.loc[df[f"MA_{period_1}"] > df[f"MA_{period_2}"], "Signal"] = pos_multiplier
        df.loc[df[f"MA_{period_1}"] <= df[f"MA_{period_2}"], "Signal"] = neg_multiplier
        
        # Calculate Strategy Returns
        df["Strat_C_Rets"], sharpe_ratio_strat = self._calculate_returns(df, False)
        
        # Get values for output
        bench_rets = df["Bench_C_Rets"].values.astype(float)
        strat_rets = df["Strat_C_Rets"].values.astype(float)
        print("Sense check: ", round(df["Close"].values[-1] / df["Close"].values[0] - 1, 3), round(bench_rets[-1], 3))
        
        # Remove irrelevant features
        if len(drop_cols) > 0:
            df = df.drop(columns=drop_cols)
        
        # Ensure Latest DF matches
        df = df.dropna()
        self.df = df
        
        # Return df
        return df, sharpe_ratio_bench, sharpe_ratio_strat

In [8]:
import numpy as np
import pandas as pd
import sklearn.mixture as mix
#from ta.momentum import RSIIndicator
import matplotlib.pyplot as plt

In [14]:
# Extract Data
start_date = "2010-01-01"
end_date = "2022-10-05"
symbol1 = "BTC-USD"
symbol2 = "ETH-USD"
imp_data1 = ImportData(symbol1, start_date, end_date)
df1 = imp_data1.df.copy()
imp_data2 = ImportData(symbol2, start_date, end_date)
df2 = imp_data2.df.copy()

In [15]:
df1.tail(10)

,Open,High,Low,Close,Volume,Returns,Range,Bench_C_Rets
Date,,,,,,,,
2022-09-24,19296.990234,19310.197266,18861.974609,18937.011719,2.614964e+10,-0.018688,0.023763,51.415439
2022-09-25,18936.310547,19134.732422,18696.468750,18802.097656,2.335997e+10,-0.007124,0.023441,51.047002
2022-09-26,18803.900391,19274.873047,18721.285156,19222.671875,4.414880e+10,0.022368,0.029570,52.208911
2022-09-27,19221.839844,20338.455078,18915.667969,19110.546875,5.857144e+10,-0.005833,0.075217,51.882143
2022-09-28,19104.621094,19688.343750,18553.296875,19426.720703,5.307130e+10,0.016544,0.061178,52.759979
2022-09-29,19427.779297,19589.265625,18924.353516,19573.050781,4.103784e+10,0.007532,0.035135,53.165975
2022-09-30,19573.431641,20109.849609,19265.662109,19431.789062,4.397525e+10,-0.007217,0.043818,52.773159
2022-10-01,19431.105469,19471.154297,19231.082031,19312.095703,1.871954e+10,-0.006160,0.012484,52.441254
2022-10-02,19311.849609,19370.308594,18970.621094,19044.107422,2.076596e+10,-0.013877,0.021069,51.699557


In [16]:
df2.tail(10)

,Open,High,Low,Close,Volume,Returns,Range,Bench_C_Rets
Date,,,,,,,,
2022-09-24,1328.244629,1346.255249,1312.640259,1317.993286,1.209821e+10,-0.007729,0.025609,4.208752
2022-09-25,1317.938477,1333.371338,1275.627686,1294.216797,1.180265e+10,-0.018040,0.045267,4.115456
2022-09-26,1294.386108,1335.526367,1282.049316,1335.320190,1.603455e+10,0.031759,0.041712,4.277986
2022-09-27,1335.337036,1396.891479,1308.991455,1330.127686,1.787060e+10,-0.003889,0.067151,4.255144
2022-09-28,1329.541382,1351.964478,1267.869263,1337.410889,1.899498e+10,0.005476,0.066328,4.284486
2022-09-29,1337.554443,1348.107666,1293.193359,1335.652344,1.379692e+10,-0.001315,0.042464,4.277515
2022-09-30,1335.646484,1368.743408,1320.383179,1327.978638,1.425010e+10,-0.005745,0.036626,4.248044
2022-10-01,1328.193726,1332.516479,1306.102539,1311.644409,6.227961e+09,-0.012300,0.020223,4.183923
2022-10-02,1311.753418,1316.330078,1275.336060,1276.093506,7.578352e+09,-0.027104,0.032144,4.043694


In [17]:
# Save DataFrame
df1.to_csv(f"data/{symbol1}.csv")

In [18]:
df2.to_csv(f"data/{symbol2}.csv")